# IA Aumentada - Técnicas de otimizacão em um problema prático

## Instalando e importando libs

In [1]:
%pip install ortools

  Using cached ortools-9.8.3296-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.8 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached protobuf-4.25.1-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached ortools-9.8.3296-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (22.9 MB)
Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
Using cached protobuf-4.25.1-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4
Note: you may need to restart the kernel to use updated packages.


In [12]:
from ortools.sat.python.cp_model import CpModel
from ortools.sat.python.cp_model import CpSolver
from ortools.sat.python.cp_model import INFEASIBLE

## Aula 1

In [4]:
class Gate:
    def __init__(self, model, k, planes_total, large, has_passaport_control):
        self.k = k
        self.large = large
        self.has_passaport_control = has_passaport_control
        self.variable = model.NewIntVar(0, planes_total, f'gate_{self.k}')
        self.neighbors = []
        self.receive_large_plane = model.NewBoolVar(f'receive_large_plane_{self.k}')
        if not self.large:
            model.Add(self.receive_large_plane == 0)

In [5]:
class Plane:
    def __init__(self, k, large, has_passaport_control):
        self.k = k
        self.large = large
        self.has_passaport_control = has_passaport_control

In [10]:
def distinct_planes(model, gates):
    variables = [gate.variable for gate in gates]
    model.AddAllDifferent(variables)

In [21]:
def all_planes_must_land(model, gates, planes_total):
    variables = {}
    for i in range (1, planes_total + 1):
        for j, gate in enumerate(gates):
            plane_i_j = model.NewBoolVar(f'plane_{i}_{j}')
            model.Add(gate.variable == i).OnlyEnforceIf(plane_i_j)
            model.Add(gate.variable != i).OnlyEnforceIf(plane_i_j.Not())
            variables[(i, j)] = plane_i_j
    for i in range(1, planes_total + 1):
        model.AddExactlyOne([variables[(i, j)] for j in range(len(gates))])

In [6]:
def large_plane_large_gate(model, gates, planes):
    large_planes = [plane for plane in planes if plane.large]
    for gate in gates:
        for plane in large_planes:
            model.Add(gate.variable != plane.k).OnlyEnforceIf(gate.receive_large_plane.Not())

In [7]:
def restrict_neighbors(model, gates, planes):
    for gate in gates:
        if not gate.large:
            continue
        for neighbor in gate.neighbors:
            if neighbor.large:
                model.Add(gate.receive_large_plane == 0).OnlyEnforceIf(neighbor.receive_large_plane)

In [17]:
def restrict_planes_with_passaport_control(model, gates, planes):
    planes_with_control = [plane for plane in planes if plane.has_passaport_control]
    gates_without_control = [gate for gate in gates if not gate.has_passaport_control]
    for gate in gates_without_control:
        for plane in planes_with_control:
            model.Add(gate.variable != plane.k)

In [25]:
def solve_model(model, gates, planes):
    solver = CpSolver()
    status = solver.Solve(model)
    print('Status: ' + solver.StatusName(status))
    if status == INFEASIBLE:
        print('Sem solucão')
        return
    for gate in gates:
        value = solver.Value(gate.variable)
        if value == 0:
            print(f'{gate.variable} sem avião')
        else:
            print(f'{gate.variable} com avião {value} | Grande: {planes[value - 1].large}')

In [22]:
planes = [Plane(1, False, True),
          Plane(2, False, False)]
planes_total = len(planes)

model = CpModel()

gates = [Gate(model, 1, planes_total, False, True),
         Gate(model, 2, planes_total, False, False)]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)
restrict_planes_with_passaport_control(model, gates, planes)

solve_model(model, gates, planes)

Status: OPTIMAL
gate_1 com avião 1 | Grande: False
gate_2 com avião 2 | Grande: False


In [23]:
planes = [Plane(1, False, False),
          Plane(2, False, True)]
planes_total = len(planes)

model = CpModel()

gates = [Gate(model, 1, planes_total, False, True),
         Gate(model, 2, planes_total, False, False)]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)
restrict_planes_with_passaport_control(model, gates, planes)

solve_model(model, gates, planes)

Status: OPTIMAL
gate_1 com avião 2 | Grande: False
gate_2 com avião 1 | Grande: False


In [26]:
planes = [Plane(1, False, True),
          Plane(2, False, True)]
planes_total = len(planes)

model = CpModel()

gates = [Gate(model, 1, planes_total, False, True),
         Gate(model, 2, planes_total, False, False)]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)
restrict_planes_with_passaport_control(model, gates, planes)

solve_model(model, gates, planes)

Status: INFEASIBLE
Sem solucão


In [27]:
planes = [Plane(1, False, True),
          Plane(2, True, False),
          Plane(3, False, False),
          Plane(4, True, True)]
planes_total = len(planes)

model = CpModel()

gates = [Gate(model, 1, planes_total, False, True),
         Gate(model, 2, planes_total, False, False),
         Gate(model, 3, planes_total, True, False),
         Gate(model, 4, planes_total, True, True)]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)
restrict_planes_with_passaport_control(model, gates, planes)

solve_model(model, gates, planes)

Status: OPTIMAL
gate_1 com avião 1 | Grande: False
gate_2 com avião 3 | Grande: False
gate_3 com avião 2 | Grande: True
gate_4 com avião 4 | Grande: True


## Aula 2

In [49]:
log_vars = []

def log_var(var):
    global log_vars
    log_vars.append(var)

In [52]:
def solve_model(model, gates, planes, penalities):
    global log_vars

    model.Minimize(sum(penalities))

    solver = CpSolver()
    status = solver.Solve(model)
    print('Status: ' + solver.StatusName(status))

    if status == INFEASIBLE:
        print('Sem solucão')
        return

    print(f'Valor Objetivo: {solver.ObjectiveValue()}')

    for var in log_vars:
        print(f'Var {var}: {solver.Value(var)}')
    log_vars = []

    for gate in gates:
        value = solver.Value(gate.variable)
        if value == 0:
            print(f'{gate.variable} sem avião')
        else:
            print(f'{gate.variable} com avião {value} | Grande: {planes[value - 1].large}')

In [41]:
def choose_if_possible_passport_control(model, gates, planes):
    penalities = []

    gates_with_control = [gate for gate in gates if gate.has_passaport_control]
    planes_without_control = [plane for plane in planes if not plane.has_passaport_control]

    for gate in gates_with_control:
        for plane in planes_without_control:
            penality = model.NewIntVar(0, 1000, f'penality_{gate.k}_{plane.k}')

            plane_in_this_gate = model.NewBoolVar(f'gate_{gate.k}_has_plane_{plane.k}')

            model.Add(gate.variable == plane.k).OnlyEnforceIf(plane_in_this_gate)
            model.Add(gate.variable != plane.k).OnlyEnforceIf(plane_in_this_gate.Not())

            model.Add(penality == 1000).OnlyEnforceIf(plane_in_this_gate)
            model.Add(penality == 0).OnlyEnforceIf(plane_in_this_gate.Not())

            log_var(penality)

            penalities.append(penality)
    
    return penalities

In [53]:
planes = [Plane(1, False, True),
          Plane(2, False, False)]
planes_total = len(planes)

model = CpModel()

gates = [Gate(model, 1, planes_total, False, True),
         Gate(model, 2, planes_total, False, False)]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)
restrict_planes_with_passaport_control(model, gates, planes)
penalities = choose_if_possible_passport_control(model, gates, planes)

solve_model(model, gates, planes, penalities)

Status: OPTIMAL
Valor Objetivo: 0.0
Var penality_1_2: 0
Var penality_1_2: 0
gate_1 com avião 1 | Grande: False
gate_2 com avião 2 | Grande: False


In [54]:
planes = [Plane(1, False, True),
          Plane(2, False, True)]
planes_total = len(planes)

model = CpModel()

gates = [Gate(model, 1, planes_total, False, True),
         Gate(model, 2, planes_total, False, False)]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)
restrict_planes_with_passaport_control(model, gates, planes)
penalities = choose_if_possible_passport_control(model, gates, planes)

solve_model(model, gates, planes, penalities)

Status: INFEASIBLE
Sem solucão


In [55]:
planes = [Plane(1, False, False),
          Plane(2, False, False)]
planes_total = len(planes)

model = CpModel()

gates = [Gate(model, 1, planes_total, False, True),
         Gate(model, 2, planes_total, False, False)]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)
restrict_planes_with_passaport_control(model, gates, planes)
penalities = choose_if_possible_passport_control(model, gates, planes)

solve_model(model, gates, planes, penalities)

Status: OPTIMAL
Valor Objetivo: 1000.0
Var penality_1_1: 0
Var penality_1_2: 1000
gate_1 com avião 2 | Grande: False
gate_2 com avião 1 | Grande: False


In [56]:
planes = [Plane(1, False, False),
          Plane(2, False, False),
          Plane(3, False, False)]
planes_total = len(planes)

model = CpModel()

gates = [Gate(model, 1, planes_total, False, True),
         Gate(model, 2, planes_total, False, False),
         Gate(model, 3, planes_total, False, True)]

distinct_planes(model, gates)
all_planes_must_land(model, gates, planes_total)
restrict_neighbors(model, gates, planes)
large_plane_large_gate(model, gates, planes)
restrict_planes_with_passaport_control(model, gates, planes)
penalities = choose_if_possible_passport_control(model, gates, planes)

solve_model(model, gates, planes, penalities)

Status: OPTIMAL
Valor Objetivo: 2000.0
Var penality_1_1: 0
Var penality_1_2: 1000
Var penality_1_3: 0
Var penality_3_1: 0
Var penality_3_2: 0
Var penality_3_3: 1000
gate_1 com avião 2 | Grande: False
gate_2 com avião 1 | Grande: False
gate_3 com avião 3 | Grande: False


## Aula 3 e 4

In [61]:
def multiply_matrix(matrix_1, matrix_2, gates_total, planes_total):
    return [matrix_1[i][j] * matrix_2[i][j]
            for i in range(planes_total)
            for j in range(gates_total)]

In [64]:
def calc_total_costs(gates_total, planes_total, distances, passengers, requeire_bus, bus_costs):
    costs = []

    for i in range(planes_total):
        plane_costs = []
        for j in range(gates_total):
            if requeire_bus[j]:
                plane_costs.append(bus_costs[1] * distances[j] * passengers[i])
            else:
                plane_costs.append(bus_costs[0] * distances[j] * passengers[i])
        costs.append(plane_costs)

    print('Custos possíveis')
    for i, row in enumerate(costs):
        print(f'Avião {i} => {row}')

    return costs

In [65]:
def define_model_elements(model, gates_total, planes_total):
    X = [[model.NewBoolVar(f'plane_{i}_gate_{j}') 
             for j in range(gates_total)] 
             for i in range(planes_total)]

    for i, row in enumerate(X):
        print(f'Avião {i} => {row}')

    return X

In [67]:
def distinct_planes(model, X, gates_total, planes_total):
    for plane_row in X:
        model.AddExactlyOne(plane_row)

    for j in range(gates_total):
        model.AddAtMostOne([X[i][j] for i in range(planes_total)])

In [57]:
def solve_model(model, X):
    solver = CpSolver()

    status = solver.Solve(model)

    if status == INFEASIBLE:
        print('Modelo sem solucão')
        return

    print(f'Status: {solver.StatusName(status)}')

    for i, row in enumerate(X):
        for j, var in enumerate(row):
            value = solver.Value(var)
            if value == 1:
                print(f'[{var}] => {value}')
            
    print(f'Custo total: {solver.ObjectiveValue()}')

In [68]:
model = CpModel()

planes_total = 1
gates_total = 3
bus_costs=(100, 500)

requeire_bus = [0, 1, 0]
distances = [10, 200, 20]
passengers = [100]

costs = calc_total_costs(gates_total, 
                         planes_total, 
                         distances, 
                         passengers, 
                         requeire_bus, 
                         bus_costs)

X = define_model_elements(model, gates_total, planes_total)

distinct_planes(model, X, gates_total, planes_total)

model.Minimize(sum(multiply_matrix(X, costs, gates_total, planes_total)))

solve_model(model, X)

Custos possíveis
Avião 0 => [100000, 10000000, 200000]
Avião 0 => [plane_0_gate_0(0..1), plane_0_gate_1(0..1), plane_0_gate_2(0..1)]
Status: OPTIMAL
[plane_0_gate_0] => 1
Custo total: 100000.0


In [69]:
model = CpModel()

planes_total = 3
gates_total = 4
bus_costs=(100, 500)

requeire_bus = [0, 1, 0, 1]
distances = [10, 200, 20, 1000]
passengers = [100, 150, 200]

costs = calc_total_costs(gates_total, 
                         planes_total, 
                         distances, 
                         passengers, 
                         requeire_bus, 
                         bus_costs)

X = define_model_elements(model, gates_total, planes_total)

distinct_planes(model, X, gates_total, planes_total)

model.Minimize(sum(multiply_matrix(X, costs, gates_total, planes_total)))

solve_model(model, X)

Custos possíveis
Avião 0 => [100000, 10000000, 200000, 50000000]
Avião 1 => [150000, 15000000, 300000, 75000000]
Avião 2 => [200000, 20000000, 400000, 100000000]
Avião 0 => [plane_0_gate_0(0..1), plane_0_gate_1(0..1), plane_0_gate_2(0..1), plane_0_gate_3(0..1)]
Avião 1 => [plane_1_gate_0(0..1), plane_1_gate_1(0..1), plane_1_gate_2(0..1), plane_1_gate_3(0..1)]
Avião 2 => [plane_2_gate_0(0..1), plane_2_gate_1(0..1), plane_2_gate_2(0..1), plane_2_gate_3(0..1)]
Status: OPTIMAL
[plane_0_gate_1] => 1
[plane_1_gate_2] => 1
[plane_2_gate_0] => 1
Custo total: 10500000.0


In [70]:
model = CpModel()

planes_total = 3
gates_total = 5
bus_costs=(100, 500)

requeire_bus = [0, 1, 0, 1, 0]
distances = [10, 200, 20, 1000, 0]
passengers = [100, 150, 200]

costs = calc_total_costs(gates_total, 
                         planes_total, 
                         distances, 
                         passengers, 
                         requeire_bus, 
                         bus_costs)

X = define_model_elements(model, gates_total, planes_total)

distinct_planes(model, X, gates_total, planes_total)

model.Minimize(sum(multiply_matrix(X, costs, gates_total, planes_total)))

solve_model(model, X)

Custos possíveis
Avião 0 => [100000, 10000000, 200000, 50000000, 0]
Avião 1 => [150000, 15000000, 300000, 75000000, 0]
Avião 2 => [200000, 20000000, 400000, 100000000, 0]
Avião 0 => [plane_0_gate_0(0..1), plane_0_gate_1(0..1), plane_0_gate_2(0..1), plane_0_gate_3(0..1), plane_0_gate_4(0..1)]
Avião 1 => [plane_1_gate_0(0..1), plane_1_gate_1(0..1), plane_1_gate_2(0..1), plane_1_gate_3(0..1), plane_1_gate_4(0..1)]
Avião 2 => [plane_2_gate_0(0..1), plane_2_gate_1(0..1), plane_2_gate_2(0..1), plane_2_gate_3(0..1), plane_2_gate_4(0..1)]
Status: OPTIMAL
[plane_0_gate_2] => 1
[plane_1_gate_0] => 1
[plane_2_gate_4] => 1
Custo total: 350000.0
